In [1]:
import napari
import numpy as np
from pathlib import Path
from tifffile import imread

from napari_cellseg3d.dev_scripts import evaluate_labels as eval
from napari_cellseg3d.utils import resize
from napari_cellseg3d.code_models.model_instance_seg import (
    binary_connected,
    binary_watershed,
    voronoi_otsu,
)

In [2]:
viewer = napari.Viewer()

In [3]:
im_path = Path("C:/Users/Cyril/Desktop/test/instance_test")
prediction_path = str(im_path / "pred.tif")
gt_labels_path = str(im_path / "labels_relabel_unique.tif")

prediction = imread(prediction_path)
gt_labels = imread(gt_labels_path)

zoom = (1 / 5, 1, 1)
prediction_resized = resize(prediction, zoom)
gt_labels_resized = resize(gt_labels, zoom)


viewer.add_image(prediction_resized, name="pred", colormap="inferno")
viewer.add_labels(gt_labels_resized, name="gt")
print(prediction_resized.shape)
print(gt_labels_resized.shape)

(25, 64, 64)
(25, 64, 64)


In [4]:
from napari_cellseg3d.utils import dice_coeff

dice_coeff(
    to_semantic(gt_labels_resized.copy()),
    to_semantic(prediction_resized.copy()),
)

0.5817600487210719

In [5]:
from napari_cellseg3d.dev_scripts.correct_labels import relabel

# gt_corrected = relabel(prediction_path, gt_labels_path, go_fast=False)

In [6]:
# eval.evaluate_model_performance(gt_labels_resized, gt_labels_resized)

In [7]:
print(prediction_resized.shape)
print(gt_labels_resized.shape)
print(np.unique(gt_labels_resized).shape[0])

(25, 64, 64)
(25, 64, 64)
125


In [8]:
connected = binary_connected(prediction_resized, thres_small=2)
viewer.add_labels(connected, name="connected")

<Labels layer 'connected' at 0x195713d1940>

In [9]:
eval.evaluate_model_performance(gt_labels_resized, connected)

2023-03-22 14:47:30,112 - Mapping labels...


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 125/125 [00:00<00:00, 3913.33it/s]

2023-03-22 14:47:30,150 - Calculating the number of neurons not found...


(65,
 46,
 13,
 12,
 0.9042297461803984,
 0.8512759824829847,
 0.9136359067720888,
 0.8728146835389444,
 1.0)

In [6]:
connected = binary_connected(prediction_resized)
viewer.add_labels(connected, name="connected")
connected.dtype

2023-03-22 15:48:47,168 - Mapping labels...


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 3454.21it/s]

2023-03-22 15:48:47,201 - Calculating the number of neurons not found...
2023-03-22 15:48:47,203 - Percent of non-fused neurons found: 54.40%
2023-03-22 15:48:47,203 - Percent of fused neurons found: 34.40%
2023-03-22 15:48:47,204 - Overall percent of neurons found: 88.80%


(68,
 43,
 13,
 10,
 0.8856947654346812,
 0.8747475859219296,
 0.9187750563205743,
 0.862012598981557,
 1.0)

In [7]:
eval.evaluate_model_performance(gt_labels_resized, connected)

2023-03-22 14:47:30,231 - Mapping labels...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 95/95 [00:00<00:00, 3069.93it/s]

2023-03-22 14:47:30,265 - Calculating the number of neurons not found...


(45,
 38,
 41,
 8,
 0.8424215218790255,
 0.9295584641244191,
 0.9332428837640889,
 0.8300682812799907,
 1.0)

In [8]:
watershed = binary_watershed(
    prediction_resized, thres_small=20, rem_seed_thres=5
)
viewer.add_labels(watershed)
eval.evaluate_model_performance(gt_labels_resized, watershed)

2023-03-22 14:47:30,344 - Mapping labels...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 80/80 [00:00<00:00, 2864.94it/s]

2023-03-22 14:47:30,376 - Calculating the number of neurons not found...


(47,
 37,
 40,
 0,
 0.8426909426266451,
 0.9355733839977192,
 0.9369165405470844,
 0.8198206611354032,
 nan)

In [9]:
voronoi = voronoi_otsu(prediction_resized, 1, outline_sigma=0.5)

from skimage.morphology import remove_small_objects

voronoi = remove_small_objects(voronoi, 10)
viewer.add_labels(voronoi)
voronoi.shape

(25, 64, 64)

In [10]:
gt_labels_resized.dtype

dtype('int64')

In [11]:
np.unique(voronoi, return_counts=True)

(array([  0,   2,   3,   7,  10,  11,  12,  14,  15,  16,  17,  18,  19,
         20,  21,  22,  23,  24,  25,  26,  27,  28,  29,  30,  31,  32,
         33,  34,  37,  38,  39,  40,  42,  43,  44,  45,  46,  47,  48,
         49,  50,  51,  52,  53,  54,  55,  56,  57,  58,  59,  60,  61,
         63,  64,  65,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,
         77,  79,  80,  81,  82,  83,  85,  86,  87,  88,  89,  90,  92,
         94,  95,  96,  97,  98,  99, 101, 102, 103, 104, 105, 106, 107,
        108, 109, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121,
        122], dtype=uint32),
 array([97911,    46,    18,    10,    47,    57,    44,    54,    22,
           94,    36,    42,    51,    41,    35,    42,    46,    47,
           52,    31,    12,    45,    68,   106,    69,    56,    32,
           69,    46,    75,    78,    41,    42,    42,    50,    50,
           48,    50,    44,    49,    50,    54,    58,    43,    41,
           39,    49,    15,    

In [12]:
np.unique(gt_labels_resized, return_counts=True)

(array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
         13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
         26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
         39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
         52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
         65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
         78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
         91,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
        105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
        118, 119, 120, 121, 122, 123, 124, 125], dtype=int64),
 array([97748,     4,     4,     4,    91,     4,    29,    54,    25,
           59,    26,    18,     4,    37,    39,    21,     4,    38,
           33,    47,    67,    31,    16,     4,    61,    92,    50,
           43,    29,    26,    38,    22,    39,    28,    32,    

In [13]:
eval.evaluate_model_performance(gt_labels_resized, voronoi)

2023-03-22 14:47:30,755 - Mapping labels...


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 105/105 [00:00<00:00, 3290.07it/s]

2023-03-22 14:47:30,792 - Calculating the number of neurons not found...


(72,
 8,
 44,
 1,
 0.8348479609766444,
 0.9314226186350036,
 0.9483750072126669,
 0.8528417100412058,
 1.0)

In [14]:
# eval.evaluate_model_performance(gt_labels_resized, voronoi)